In [1]:
pip install PyPortfolioOpt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 199.2 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 384.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 3.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 218.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 kB 135.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 15.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Install required packages if not already installed:
# pip install yfinance PyPortfolioOpt

import yfinance as yf
import pandas as pd
from pypfopt import EfficientFrontier, risk_models, expected_returns

# Define a list of 50 stocks (example tickers from S&P indices)
tickers = [
    "AAPL", "MSFT", "AMZN", "GOOG", "GOOGL", "META", "TSLA", "BRK-B", "JPM", "JNJ",
    "V", "PG", "NVDA", "UNH", "HD", "MA", "DIS", "PYPL", "BAC", "VZ",
    "ADBE", "CMCSA", "NFLX", "T", "INTC", "CSCO", "PFE", "CRM", "ABT", "TMO",
    "CVX", "MRK", "ORCL", "KO", "ACN", "LLY", "MCD", "NKE", "WMT", "NEE",
    "DHR", "IBM", "TXN", "COST", "LIN", "SBUX", "PM", "LOW", "QCOM", "BMY"
]

# Download historical adjusted close prices.
# Here we download data from January 1, 2020 to January 1, 2025 (adjust as needed).
data = yf.download(tickers, start="2020-01-01", end="2025-01-01")["Close"]

# Drop any columns with missing data (optional but recommended)
data = data.dropna(axis=1)

# Calculate the expected returns and sample covariance matrix.
mu = expected_returns.mean_historical_return(data)
S = risk_models.sample_cov(data)

# Initialize the Efficient Frontier with weight bounds between 0 and 1.
ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))

# Optimize for maximum Sharpe ratio.
weights = ef.max_sharpe()

# Clean the weights (rounding small numbers to zero, etc.)
cleaned_weights = ef.clean_weights()

print("Optimized Weights:")
print(cleaned_weights)

# Get the expected portfolio performance: expected annual return, volatility, and Sharpe ratio.
performance = ef.portfolio_performance(verbose=True)

[*********************100%***********************]  50 of 50 completed

Optimized Weights:
OrderedDict([('AAPL', 0.0), ('ABT', 0.0), ('ACN', 0.0), ('ADBE', 0.0), ('AMZN', 0.0), ('BAC', 0.0), ('BMY', 0.0), ('BRK-B', 0.0), ('CMCSA', 0.0), ('COST', 0.0), ('CRM', 0.0), ('CSCO', 0.0), ('CVX', 0.0), ('DHR', 0.0), ('DIS', 0.0), ('GOOG', 0.0), ('GOOGL', 0.0), ('HD', 0.0), ('IBM', 0.0), ('INTC', 0.0), ('JNJ', 0.0), ('JPM', 0.0), ('KO', 0.0), ('LIN', 0.0), ('LLY', 0.43416), ('LOW', 0.0), ('MA', 0.0), ('MCD', 0.0), ('META', 0.0), ('MRK', 0.0), ('MSFT', 0.0), ('NEE', 0.0), ('NFLX', 0.0), ('NKE', 0.0), ('NVDA', 0.29806), ('ORCL', 0.0), ('PFE', 0.0), ('PG', 0.0), ('PM', 0.0), ('PYPL', 0.0), ('QCOM', 0.0), ('SBUX', 0.0), ('T', 0.0), ('TMO', 0.0), ('TSLA', 0.09206), ('TXN', 0.0), ('UNH', 0.0), ('V', 0.0), ('VZ', 0.0), ('WMT', 0.17572)])
Expected annual return: 55.0%
Annual volatility: 28.5%
Sharpe Ratio: 1.93


In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp  # Import CVXPY
from pypfopt import EfficientFrontier, risk_models, expected_returns

# Define a list of 50 tickers (note: 'FB' has been updated to 'META')
tickers = [
    "AAPL", "MSFT", "AMZN", "GOOG", "GOOGL", "META", "TSLA", "BRK-B", "JPM", "JNJ",
    "V", "PG", "NVDA", "UNH", "HD", "MA", "DIS", "PYPL", "BAC", "VZ",
    "ADBE", "CMCSA", "NFLX", "T", "INTC", "CSCO", "PFE", "CRM", "ABT", "TMO",
    "CVX", "MRK", "ORCL", "KO", "ACN", "LLY", "MCD", "NKE", "WMT", "NEE",
    "DHR", "IBM", "TXN", "COST", "LIN", "SBUX", "PM", "LOW", "QCOM", "BMY"
]

# Download historical adjusted close prices.
data = yf.download(tickers, start="2020-01-01", end="2025-01-01")["Close"]

# Remove any columns with missing data.
data = data.dropna(axis=1)

# Calculate expected returns and the sample covariance matrix.
mu = expected_returns.mean_historical_return(data)
S = risk_models.sample_cov(data)

# Initialize the Efficient Frontier with weight bounds between 0 and 1 (no short-selling).
ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))

# Define the L2 regularisation coefficient.
l2_reg = 10

# Add the L2 regularisation objective using cvxpy.sum_squares
ef.add_objective(lambda w: l2_reg * cp.sum_squares(w))

# Optimize for maximum Sharpe ratio.
weights = ef.max_sharpe()

# Clean the weights (round very small values to zero, etc.)
cleaned_weights = ef.clean_weights()

print("Optimized Weights with L2 Regularisation:")
print(cleaned_weights)

# Get and print the expected portfolio performance: annual return, volatility, and Sharpe ratio.
performance = ef.portfolio_performance(verbose=True)

[*********************100%***********************]  50 of 50 completed

Optimized Weights with L2 Regularisation:
OrderedDict([('AAPL', 0.03985), ('ABT', 0.00833), ('ACN', 0.01527), ('ADBE', 0.00244), ('AMZN', 0.02373), ('BAC', 0.0064), ('BMY', 0.0), ('BRK-B', 0.0207), ('CMCSA', 0.0), ('COST', 0.04198), ('CRM', 0.01786), ('CSCO', 0.00763), ('CVX', 0.0097), ('DHR', 0.01424), ('DIS', 0.0), ('GOOG', 0.03145), ('GOOGL', 0.03114), ('HD', 0.0195), ('IBM', 0.0238), ('INTC', 0.0), ('JNJ', 0.00231), ('JPM', 0.01927), ('KO', 0.00692), ('LIN', 0.02271), ('LLY', 0.06938), ('LOW', 0.02361), ('MA', 0.01479), ('MCD', 0.01338), ('META', 0.03026), ('MRK', 0.00728), ('MSFT', 0.03041), ('NEE', 0.00653), ('NFLX', 0.02969), ('NKE', 0.0), ('NVDA', 0.13134), ('ORCL', 0.03998), ('PFE', 0.0), ('PG', 0.01234), ('PM', 0.01882), ('PYPL', 0.0), ('QCOM', 0.01565), ('SBUX', 0.0), ('T', 0.00186), ('TMO', 0.01238), ('TSLA', 0.10345), ('TXN', 0.01163), ('UNH', 0.01809), ('V', 0.01385), ('VZ', 0.0), ('WMT', 0.03006)])
Expected annual return: 34.2%
Annual volatility: 25.4%
Sharpe Ratio: 1.35


/usr/local/lib/python3.11/dist-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(
